In [1]:
# from matplotlib import pyplot as plt

# H = nx.gaussian_random_partition_graph(n=100,s=10,v=10,p_in=.25,p_out=.1)
# H = nx.relaxed_caveman_graph(l=3,k=15,p=0.15)
# H= nx.powerlaw_cluster_graph(n=200,m=10,p=0.2)
# H = nx.geographical_threshold_graph(200, 60)

# nx.draw(H)
# plt.figure(2,figsize=(20,20))
# nx.draw_spring(H,node_size=100,font_size=8) 
# plt.show()

In [2]:
# get the files from model.py
import math
from enum import Enum
import networkx as nx
from random import randrange

from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
from mesa.space import NetworkGrid

# Model

In [3]:
#### THIS CAN BE SUBSTITUTED WITH BIAS ##########
#Define the state of a new article
class State(Enum):
    NEUTRAL = 0 #SUSCEPTIBLE = 0
    FAKE = 1 #INFECTED = 1
    RESISTANT = 2

In [4]:
def number_state(model, state):
    """
    GET THE TOTAL NUMBER OF PEOPLE FOR THE STATE YOU ARE LOOKING FOR
    
    sudo code
    for person in model.grid.get_all_cell_contents():
        if person.state is state:
            add 1 to the sum
    """
    return sum([1 for a in model.grid.get_all_cell_contents() if a.state is state])


def number_spammers(model):
    """
    use the number_state function to get back the total number of people who passed FAKE news
    """
    return number_state(model, State.FAKE)


def number_neutral(model):
    """
    use the number_state function to get back the total number of people who have not yet passed fake news
    """
    return number_state(model, State.NEUTRAL)


def number_resistant(model):
    """
    use the number_state function to get back the total number of people RESISTANT to FAKE news
    """
    return number_state(model, State.RESISTANT)

In [6]:
class SocialNetwork(Model):
    """A social network model with some number of agents"""
    
    """MODEL CONSTRUCTOR NEEDS AN __init__ method and step method"""
    def __init__(
        self,
        num_nodes=90,
        avg_node_degree=20,
        initial_outbreak_size=2,
        ##### think about changing the ones below ###############
        fake_news_spread_chance=0.4,
        fake_news_check_frequency=0.4,
        recovery_chance=0.3,
        gain_resistance_chance=0.5,
        #########################################################
    ):

        #INITIALIZE THE NETWORK, CHANGE HERE TO OTHER GRAPH TYPES
        self.num_nodes = num_nodes
        prob = avg_node_degree / self.num_nodes
#         self.G = nx.erdos_renyi_graph(n=self.num_nodes, p=prob)
#         self.G = nx.geographical_threshold_graph(n=self.num_nodes,theta = prob*self.num_nodes)

        """
        Relaxed Caveman Graph allows to create small communities that are connected. Its parameters are:
        l (int) – Number of groups, which should be a fraction of num_nodes
        k (int) – Size of cliques
        p (float) – Probabilty of rewiring each edge.
        """
#         l = randrange(int(math.log1p(self.num_nodes)),int(math.log1p(self.num_nodes)+0.07*self.num_nodes)) #lnx to lnx + 0.07*x
#         k = self.num_nodes//l
#         self.G = nx.relaxed_caveman_graph(l=l,k=k,p=prob)
        
        """
        Powerlaw Cluster Graph creates a graph according to the power law distribution and follows Holme and Kim algorithm:
        n (int) – the number of nodes
        m (int) – the number of random edges to add for each new node
        p (float,) – Probability of adding a triangle after adding a random edge
        seed (int, optional) – Seed for random number generator (default=None).
        """
        self.G = nx.powerlaw_cluster_graph(n=self.num_nodes,m=avg_node_degree, p = prob)
        
        #initialize the rest of the model attributes
        self.grid = NetworkGrid(self.G)
        self.schedule = RandomActivation(self)
        self.initial_outbreak_size = (
            initial_outbreak_size if initial_outbreak_size <= num_nodes else num_nodes
        )
        self.fake_news_spread_chance = fake_news_spread_chance
        self.fake_news_check_frequency = fake_news_check_frequency
        self.recovery_chance = recovery_chance
        self.gain_resistance_chance = gain_resistance_chance

        #initialize the datacollector
        self.datacollector = DataCollector(
            {
                "Uers Spreading Fake News": number_spammers,
                "Casual Users": number_neutral,
                "Resistant": number_resistant,
            }
        )
        ##END CONSTRUCTOR

        # HERE WE BUILD THE NETWORK
        # Create agents
        for i, node in enumerate(self.G.nodes()):
            #make an agent
            a = User(
                i,
                self,
                State.NEUTRAL, # HERE WE CAN RANDOMLY PICK BIAS
                self.fake_news_spread_chance,
                self.fake_news_check_frequency,
                self.recovery_chance,
                self.gain_resistance_chance,
            )
            self.schedule.add(a)
            # Add the agent to the node
            self.grid.place_agent(a, node)

        # Infect some nodes
        infected_nodes = self.random.sample(self.G.nodes(), self.initial_outbreak_size)
        for a in self.grid.get_cell_list_contents(infected_nodes):
            a.state = State.FAKE

        self.running = True
        self.datacollector.collect(self)

    def resistant_susceptible_ratio(self):
        try:
            return number_state(self, State.RESISTANT) / number_state(
                self, State.NEUTRAL
            )
        except ZeroDivisionError:
            return math.inf

    def step(self):
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)

    def run_model(self, n):
        for i in range(n):
            self.step()

In [7]:
class User(Agent):
    # THIS COULD BE THE USER
    
    #CONSTRUCTOR -- REPLACE WITH USER ATTRIBUTES
    def __init__(
        self,
        unique_id,
        model,
        initial_state,
        fake_news_spread_chance,
        fake_news_check_frequency,
        recovery_chance,
        gain_resistance_chance,
    ):
        super().__init__(unique_id, model)

        self.state = initial_state

        self.fake_news_spread_chance = fake_news_spread_chance
        self.fake_news_check_frequency = fake_news_check_frequency
        self.recovery_chance = recovery_chance
        self.gain_resistance_chance = gain_resistance_chance

    def try_to_infect_neighbors(self):
        # THE WHOLE THING IS RANDOM
        neighbors_nodes = self.model.grid.get_neighbors(self.pos, include_center=False)
        susceptible_neighbors = [
            agent
            for agent in self.model.grid.get_cell_list_contents(neighbors_nodes)
            if agent.state is State.NEUTRAL
        ]
        for a in susceptible_neighbors:
            if self.random.random() < self.fake_news_spread_chance:
                a.state = State.FAKE

    def try_gain_resistance(self):
        # THE WHOLE THING IS RANDOM
        if self.random.random() < self.gain_resistance_chance:
            self.state = State.RESISTANT

    def try_remove_infection(self):
        # Try to remove
        if self.random.random() < self.recovery_chance:
            # Success
            self.state = State.NEUTRAL
            self.try_gain_resistance()
        else:
            # Failed
            self.state = State.FAKE

    def try_check_situation(self):
        if self.random.random() < self.fake_news_check_frequency:
            # Checking...
            if self.state is State.FAKE:
                self.try_remove_infection()

    def step(self):
        if self.state is State.FAKE:
            self.try_to_infect_neighbors()
        self.try_check_situation()

# Server

In [8]:
# get the files from server.py
import math

from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter
from mesa.visualization.modules import ChartModule
from mesa.visualization.modules import NetworkModule
from mesa.visualization.modules import TextElement
#from .model import SocialNetwork, State, number_infected

In [9]:
def network_portrayal(G):
    # The model ensures there is always 1 agent per node

    def node_color(agent):
        return {State.FAKE: "#FF0000", State.NEUTRAL: "#008000"}.get(
            agent.state, "#808080"
        )

    def edge_color(agent1, agent2):
        if State.RESISTANT in (agent1.state, agent2.state):
            return "#000000"
        return "#cfcfcf"

    def edge_width(agent1, agent2):
        if State.RESISTANT in (agent1.state, agent2.state):
            return 3
        return 2

    def get_agents(source, target):
        return G.nodes[source]["agent"][0], G.nodes[target]["agent"][0]

    portrayal = dict()
    portrayal["nodes"] = [
        {
            "size": 6,
            "color": node_color(agents[0]),
            "tooltip": "id: {}<br>state: {}".format(
                agents[0].unique_id, agents[0].state.name
            ),
        }
        for (_, agents) in G.nodes.data("agent")
    ]

    portrayal["edges"] = [
        {
            "source": source,
            "target": target,
            "color": edge_color(*get_agents(source, target)),
            "width": edge_width(*get_agents(source, target)),
        }
        for (source, target) in G.edges
    ]

    return portrayal


network = NetworkModule(network_portrayal, 500, 500, library="d3")
chart = ChartModule(
    [
        {"Label": "Spammer", "Color": "#FF0000"},
        {"Label": "Neutral", "Color": "#008000"},
        {"Label": "Resistant", "Color": "#808080"},
    ]
)


class MyTextElement(TextElement):
    def render(self, model):
        ratio = model.resistant_susceptible_ratio()
        ratio_text = "&infin;" if ratio is math.inf else "{0:.2f}".format(ratio)
        infected_text = str(number_spammers(model))

        return "Resistant/Neutral Ratio: {}<br>Infected Remaining: {}".format(
            ratio_text, infected_text
        )


model_params = {
    "num_nodes": UserSettableParameter(
        "slider",
        "Number of agents",
        10,
        10,
        100,
        1,
        description="Choose how many agents to include in the model",
    ),
    "avg_node_degree": UserSettableParameter(
        "slider", "Avg Node Degree", 3, 3, 8, 1, description="Avg Node Degree"
    ),
    "initial_outbreak_size": UserSettableParameter(
        "slider",
        "Initial Outbreak Size",
        1,
        1,
        10,
        1,
        description="Initial Outbreak Size",
    ),
    "fake_news_spread_chance": UserSettableParameter(
        "slider",
        "Fake News Spread Chance",
        0.4,
        0.0,
        1.0,
        0.1,
        description="Probability that susceptible neighbor will be infected",
    ),
    "fake_news_check_frequency": UserSettableParameter(
        "slider",
        "Fake News Check Frequency",
        0.4,
        0.0,
        1.0,
        0.1,
        description="Frequency the nodes check whether they are spreading " " fake news",
    ),
    "recovery_chance": UserSettableParameter(
        "slider",
        "Recovery Chance",
        0.3,
        0.0,
        1.0,
        0.1,
        description="Probability that the fake news will be removed",
    ),
    "gain_resistance_chance": UserSettableParameter(
        "slider",
        "Gain Resistance Chance",
        0.5,
        0.0,
        1.0,
        0.1,
        description="Probability that a recovered agent will become "
        "resistant to fake news in the future",
    ),
}

server = ModularServer(
    SocialNetwork, [network, MyTextElement(), chart], "Fake News Propogation Model", model_params
)
server.port = 8521

In [10]:
server.launch()

Interface starting at http://127.0.0.1:8521
Socket opened!
{"type":"get_params"}
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"get_step","step":27}
{"type":"get_step","step":28}
{"type":"get_step","step":29}
{"type":"get_step","step":30}
{"type":"ge